In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

2025-02-09 15:49:30.588065: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-09 15:49:30.677141: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-09 15:49:30.755505: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739096370.826939    5702 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739096370.847278    5702 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-09 15:49:31.019676: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
df = pd.read_csv("insurance.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],
                                                    df.bought_insurance, test_size=0.2, random_state=25)

In [ ]:
X_train

In [12]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [ ]:
X_train_scaled

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,),
                       activation='sigmoid',
                       kernel_initializer='ones',
                       bias_initializer='zeros')
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X_train_scaled, y_train, epochs = 5000)

In [16]:
model.evaluate (X_test_scaled, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 1.0000 - loss: 0.3550


[0.35497763752937317, 1.0]

In [17]:
model.predict(X_test_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


array([[0.70548475],
       [0.3556957 ],
       [0.16827832],
       [0.4780119 ],
       [0.72606957],
       [0.82949823]], dtype=float32)

In [26]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [20]:
coef, intercept = model.get_weights()
coef, intercept

(array([[5.0608625],
        [1.408652 ]], dtype=float32),
 array([-2.9137027], dtype=float32))

# Using obtained weights and bias, manually doing prediction -- 

In [27]:
def sigmoid(x):
    import math
    return 1/(1 + math.exp(-x))

In [28]:
def prediction_function(age, affordibility):
    weighted_Sum = coef[0]*age + coef[1]*affordibility + intercept
    return sigmoid(weighted_Sum)

In [33]:
prediction_function(0.22, 1)

/tmp/ipykernel_5702/86937764.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1/(1 + math.exp(-x))


0.4033175299485961

# Replicating the Tensorflow model using mathematical calculations.

In [34]:
def logloss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i, epsilon) for i in y_predicted]
    y_predicted_new = [min(i, 1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [35]:
def sigmoid_numpy(X):
    return 1/(1+np.exp(-X))


In [58]:
def gradient_descent(age, affordibility, y_true, epochs, loss_threshold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    
    for i in range(epochs):
        weighted_sum = w1*age + w2*affordibility + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = logloss(y_true, y_predicted)
        
        if loss<=loss_threshold:
            break
        
        w1d = (1/n)*np.dot(np.transpose(age), (y_predicted-y_true))
        w2d = (1/n)*np.dot(np.transpose(affordibility), (y_predicted-y_true))
        bias_d = np.mean(y_predicted-y_true)

        w1 = w1 - rate*w1d
        w2 = w2 - rate*w2d
        bias = bias - rate*bias_d

        #print(f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
    
    return w1, w2, bias, loss

In [59]:
gradient_descent(X_train_scaled['age'], X_train_scaled['affordibility'],y_train,1000, 0.4631)

(5.042801249636176,
 1.4567027197201048,
 -2.956045609784402,
 0.46293944095888917)

# Observe the Tensorflow generated weights & bias nearly match with manual calculations 

In [54]:
coef, intercept

(array([[5.0608625],
        [1.408652 ]], dtype=float32),
 array([-2.9137027], dtype=float32))